In [ ]:
import os

from cartopy import crs
import distributed
from matplotlib import pyplot as plt
import pandas as pd
import xarray as xr

_ = xr.set_options(keep_attrs=True)

In [ ]:
# use a large filesystem for cahce
#
# os.chdir("/data")

# activate dask distributed
#
# client = distributed.Client("tcp://dask-scheduler:8786")
# client

In [ ]:
# !pip install ecmwf-opendata 
# !pip install -e ../

In [ ]:
request = {
    "dataset": "reanalysis-era5-land",
    "variable": ["total_precipitation"],
    "date": ["2022-01-01/2023-07-31"],
    "time": [f"{h:02}:00" for h in range(24)],
    "area": ["E"],
}

In [ ]:
%%time

ds = xr.open_dataset(
    request,
    engine="ecmwf",
    request_chunks={"day": 10},
    cache_kwargs={"cache_file": True},
    chunks={},
)
ds

In [ ]:
# visualisation

italy = dict(lat=slice(35, 47), lon=slice(6, 19))
vmax = 0.13
projection = crs.EuroPP()
cmap = "Blues"

In [ ]:
%%time

f, ax = plt.subplots(subplot_kw={"projection": projection})
ds.total_precipitation.sel(time=["2023-05-16T18:00", "2023-05-16T19:00"]).diff("time").plot(ax=ax, cmap=cmap, vmax=vmax/24, transform=crs.PlateCarree())
_ = ax.coastlines()

In [ ]:
%%time

f, ax = plt.subplots(subplot_kw={"projection": projection})
ds.total_precipitation.sel(time=["2023-05-17T00:00", "2023-05-18T00:00"]).sum("time").plot(ax=ax, cmap=cmap, vmax=vmax, transform=crs.PlateCarree())
_ = ax.coastlines()

In [ ]:
%%time

f, ax = plt.subplots(subplot_kw={"projection": projection})
ds.total_precipitation.sel(time=pd.date_range("2022-05-01", "2022-05-31", freq="D")).sum("time").plot(ax=ax, cmap=cmap, vmax=vmax, transform=crs.PlateCarree())
_ = ax.coastlines()